In [1]:
!pip install geemap
!pip install eerepr
!pip install geopandas
import os,tarfile
import geemap
import subprocess
import ee
import eerepr
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl
import numpy as np
import pandas as pd
import folium
import copy
import math
from tqdm import trange,tqdm
from functools import partial
import geopandas as gpd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 115.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.8 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-non

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=P7n9HSt5GjehOKzUdqyKFR0N_HMsKC-TXbHat510dPw&tc=rqdF9ISaiqJfqKUyzubJCWDcECR4pCwQ6zP17fTBGbc&cc=e7HoUYq_ICYfUkDsgYp2vB2rBAdYYZH1WDUnkSPuICI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VO2PP7qrYDM4tx1TRjUIS3P7ePXcS3YcdAuPFZUk2Ud6nxxiTK_PHs

Successfully saved authorization token.


In [3]:
Glacial_lake_2015A = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/Glacial_lake/Wu_Asia_Southest_GL_wgs84').filter(ee.Filter.gte('GL_Area', 0.1))  
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()
})
Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())

Glacial_lake_2015C = Glacial_lake_2015A.map(Geo_ext).map(Centrid_set)  
Glacial_lake_2015R = Glacial_lake_2015A.map(Rectangle_set)       

Glacial_lake_2015A_GeoList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(), ['Geo']).get('list'))
Glacial_lake_2015C_CentriodList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_2015R_RectangleList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))
Num_list = Glacial_lake_2015C_CentriodList.size().getInfo()

In [4]:
#-----------------------------------------------地形校正-----------------------------------------
def slope_correction(collection, elevation, model, buffer=0):
    '''This function applies the slope correction on a collection of Sentinel-1 data
       
       :param collection: ee.Collection or ee.Image of Sentinel-1
       :param elevation: ee.Image of DEM
       :param model: model to be applied (volume/surface)
       :param buffer: buffer in meters for layover/shadow amsk
        
       :returns: ee.Image
    '''
    def _volumetric_model_SCF(theta_iRad, alpha_rRad):
        '''Code for calculation of volumetric model SCF
            体积模型
        :param theta_iRad: ee.Image of incidence angle in radians
        :param alpha_rRad: ee.Image of slope steepness in range
        
        :returns: ee.Image
        '''

        # create a 90 degree image in radians
        ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)

        # model
        nominator = (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).tan()
        denominator = (ninetyRad.subtract(theta_iRad)).tan()
        return nominator.divide(denominator)

    def _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad):
        '''Code for calculation of direct model SCF
            表面模型
        :param theta_iRad: ee.Image of incidence angle in radians
        :param alpha_rRad: ee.Image of slope steepness in range
        :param alpha_azRad: ee.Image of slope steepness in azimuth
        
        :returns: ee.Image
        '''

        # create a 90 degree image in radians
        ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)

        # model
        nominator = (ninetyRad.subtract(theta_iRad)).cos()
        denominator = (alpha_azRad.cos().multiply(
            (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).cos()))

        return nominator.divide(denominator)

    def _erode(image, distance):
        '''Buffer function for raster
            腐蚀算法，输入的图像需要额外的缓冲
        :param image: ee.Image that shoudl be buffered
        :param distance: distance of buffer in meters

        :returns: ee.Image
        '''

        d = (image.Not().unmask(1).fastDistanceTransform(30).sqrt().multiply(
            ee.Image.pixelArea().sqrt()))

        return image.updateMask(d.gt(distance))

    def _masking(alpha_rRad, theta_iRad, buffer):
        '''Masking of layover and shadow
            获取几何畸变区域
        :param alpha_rRad: ee.Image of slope steepness in range
        :param theta_iRad: ee.Image of incidence angle in radians
        :param buffer: buffer in meters
        
        :returns: ee.Image
        '''
        # layover, where slope > radar viewing angle
        layover = alpha_rRad.lt(theta_iRad).rename('layover')

        # shadow
        ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)
        shadow = alpha_rRad.gt(
            ee.Image.constant(-1).multiply(
                ninetyRad.subtract(theta_iRad))).rename('shadow')

        # add buffer to layover and shadow
        if buffer > 0:
            layover = _erode(layover, buffer)
            shadow = _erode(shadow, buffer)

        # combine layover and shadow
        no_data_mask = layover.And(shadow).rename('no_data_mask')

        return layover.addBands(shadow).addBands(no_data_mask)

    def _correct(image):
        '''This function applies the slope correction and adds layover and shadow masks
        
        '''

        # get the image geometry and projection
        geom = image.geometry()
        proj = image.select(1).projection()

        # calculate the look direction
        heading = (ee.Terrain.aspect(image.select('angle')).reduceRegion(
            ee.Reducer.mean(), geom, 1000).get('aspect'))

        # Sigma0 to Power of input image，Sigma0是指雷达回波信号的强度
        sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))

        # the numbering follows the article chapters
        # 2.1.1 Radar geometry
        theta_iRad = image.select('angle').multiply(np.pi / 180)
        phi_iRad = ee.Image.constant(heading).multiply(np.pi / 180)

        # 2.1.2 Terrain geometry
        alpha_sRad = ee.Terrain.slope(elevation).select('slope').multiply(
            np.pi / 180).setDefaultProjection(proj).clip(geom)
        phi_sRad = ee.Terrain.aspect(elevation).select('aspect').multiply(
            np.pi / 180).setDefaultProjection(proj).clip(geom)

        # we get the height, for export
        height = elevation.setDefaultProjection(proj).clip(geom)

        # 2.1.3 Model geometry
        #reduce to 3 angle
        phi_rRad = phi_iRad.subtract(phi_sRad)

        # slope steepness in range (eq. 2)
        alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()

        # slope steepness in azimuth (eq 3)
        alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()

        # local incidence angle (eq. 4)
        theta_liaRad = (alpha_azRad.cos().multiply(
            (theta_iRad.subtract(alpha_rRad)).cos())).acos()
        theta_liaDeg = theta_liaRad.multiply(180 / np.pi)

        # 2.2
        # Gamma_nought
        gamma0 = sigma0Pow.divide(theta_iRad.cos())
        gamma0dB = ee.Image.constant(10).multiply(gamma0.log10()).select(
            ['VV', 'VH'], ['VV_gamma0', 'VH_gamma0'])
        ratio_gamma = (gamma0dB.select('VV_gamma0').subtract(
            gamma0dB.select('VH_gamma0')).rename('ratio_gamma0'))

        if model == 'volume':
            scf = _volumetric_model_SCF(theta_iRad, alpha_rRad)

        if model == 'surface':
            scf = _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad)

        # apply model for Gamm0_f
        gamma0_flat = gamma0.divide(scf)
        gamma0_flatDB = (ee.Image.constant(10).multiply(
            gamma0_flat.log10()).select(['VV', 'VH'],
                                        ['VV_gamma0flat', 'VH_gamma0flat']))

        masks = _masking(alpha_rRad, theta_iRad, buffer)

        # calculate the ratio for RGB vis
        ratio_flat = (gamma0_flatDB.select('VV_gamma0flat').subtract(
            gamma0_flatDB.select('VH_gamma0flat')).rename('ratio_gamma0flat'))

        return (image.rename(['VV_sigma0', 'VH_sigma0','incAngle'])
                .addBands(gamma0dB)
                .addBands(ratio_gamma)
                .addBands(gamma0_flatDB)
                .addBands(ratio_flat)
                .addBands(alpha_rRad.rename('alpha_rRad'))
                .addBands(alpha_azRad.rename('alpha_azRad'))
                .addBands(phi_sRad.rename('aspect'))
                .addBands(alpha_sRad.rename('slope'))
                .addBands(theta_iRad.rename('theta_iRad'))
                .addBands(theta_liaRad.rename('theta_liaRad'))
                .addBands(masks).addBands(height.rename('elevation')))

    # run and return correction
    if type(collection) == ee.imagecollection.ImageCollection:
        return collection.map(_correct)
    elif type(collection) == ee.image.Image:
        return _correct(collection)
    else:
        print('Check input type, only image collection or image can be input')


#-----------------------------------------------图像滤波-----------------------------------------
def boxcar(KERNEL_SIZE):
    def wrap(image):
        bandNames = image.bandNames().remove('angle')
        #Define a boxcar kernel
        kernel = ee.Kernel.square((KERNEL_SIZE/2), units='pixels', normalize=True)
        #Apply boxcar
        output = image.select(bandNames).convolve(kernel).rename(bandNames)
        return image.addBands(output, None, True)
    return wrap

def leefilter(KERNEL_SIZE):
    def wrap(image):
        bandNames = image.bandNames().remove('angle')
        # S1-GRD images are multilooked 5 times in range
        enl = 5
        # Compute the speckle standard deviation
        eta = 1.0/math.sqrt(enl)
        eta = ee.Image.constant(eta)
        # MMSE estimator
        # Neighbourhood mean and variance
        oneImg = ee.Image.constant(1)
        # Estimate stats
        reducers = ee.Reducer.mean().combine(
                      reducer2= ee.Reducer.variance()
                      ,sharedInputs= True
                      )
        stats = (image.select(bandNames).reduceNeighborhood(
                      reducer= reducers
                          ,kernel= ee.Kernel.square(KERNEL_SIZE/2, 'pixels')
                              ,optimization= 'window'))
        meanBand = bandNames.map(lambda bandName: ee.String(bandName).cat('_mean'))
        varBand = bandNames.map(lambda bandName:  ee.String(bandName).cat('_variance'))

        z_bar = stats.select(meanBand)
        varz = stats.select(varBand)
        # Estimate weight 
        varx = (varz.subtract(z_bar.pow(2).multiply(eta.pow(2)))).divide(oneImg.add(eta.pow(2)))
        b = varx.divide(varz)
        # if b is negative set it to zero
        new_b = b.where(b.lt(0), 0)
        output = oneImg.subtract(new_b).multiply(z_bar.abs()).add(new_b.multiply(image.select(bandNames)))
        output = output.rename(bandNames)
        return image.addBands(output, None, True)
    return wrap    

def gammamap(KERNEL_SIZE): 
    def wrap(image):
        enl = 5
        bandNames = image.bandNames().remove('angle')
        #local mean
        reducers = ee.Reducer.mean().combine( \
                      reducer2= ee.Reducer.stdDev(), \
                      sharedInputs= True
                      )
        stats = (image.select(bandNames).reduceNeighborhood( \
                      reducer= reducers, \
                          kernel= ee.Kernel.square(KERNEL_SIZE/2,'pixels'), \
                              optimization= 'window'))
        meanBand = bandNames.map(lambda bandName: ee.String(bandName).cat('_mean'))
        stdDevBand = bandNames.map(lambda bandName:  ee.String(bandName).cat('_stdDev'))
        z = stats.select(meanBand)
        sigz = stats.select(stdDevBand)
        #local observed coefficient of variation
        ci = sigz.divide(z)
        #noise coefficient of variation (or noise sigma)
        cu = 1.0/math.sqrt(enl)
        #threshold for the observed coefficient of variation
        cmax = math.sqrt(2.0) * cu
        cu = ee.Image.constant(cu)
        cmax = ee.Image.constant(cmax)
        enlImg = ee.Image.constant(enl)
        oneImg = ee.Image.constant(1)
        twoImg = ee.Image.constant(2)
        alpha = oneImg.add(cu.pow(2)).divide(ci.pow(2).subtract(cu.pow(2)))
        #Implements the Gamma MAP filter described in equation 11 in Lopez et al. 1990
        q = image.select(bandNames).expression('z**2 * (z * alpha - enl - 1)**2 + 4 * alpha * enl * b() * z', { 'z': z,  'alpha':alpha,'enl': enl})
        rHat = z.multiply(alpha.subtract(enlImg).subtract(oneImg)).add(q.sqrt()).divide(twoImg.multiply(alpha))
        #if ci <= cu then its a homogenous region ->> boxcar filter
        zHat = (z.updateMask(ci.lte(cu))).rename(bandNames)
        #if cmax > ci > cu then its a textured medium ->> apply Gamma MAP filter
        rHat = (rHat.updateMask(ci.gt(cu)).updateMask(ci.lt(cmax))).rename(bandNames)
        #ci>cmax then its strong signal ->> retain
        x = image.select(bandNames).updateMask(ci.gte(cmax)).rename(bandNames)  
        #Merge
        output = ee.ImageCollection([zHat,rHat,x]).sum()
        return image.addBands(output, None, True)
    return wrap
       
def RefinedLee(image):

    bandNames = image.bandNames().remove('angle')

    def inner(b):

        img = image.select([b]);
    
        # img must be linear, i.e. not in dB!
        # Set up 3x3 kernels 
        weights3 = ee.List.repeat(ee.List.repeat(1,3),3);
        kernel3 = ee.Kernel.fixed(3,3, weights3, 1, 1, False);
  
        mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3);
        variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3);
  
        # Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
        sample_weights = ee.List([[0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0], [0,1,0,1,0,1,0], [0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0]]);
  
        sample_kernel = ee.Kernel.fixed(7,7, sample_weights, 3,3, False);
  
        # Calculate mean and variance for the sampled windows and store as 9 bands
        sample_mean = mean3.neighborhoodToBands(sample_kernel); 
        sample_var = variance3.neighborhoodToBands(sample_kernel);
  
        # Determine the 4 gradients for the sampled windows
        gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs();
        gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs());
        gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs());
        gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs());
  
        # And find the maximum gradient amongst gradient bands
        max_gradient = gradients.reduce(ee.Reducer.max());
  
        # Create a mask for band pixels that are the maximum gradient
        gradmask = gradients.eq(max_gradient);
  
        # duplicate gradmask bands: each gradient represents 2 directions
        gradmask = gradmask.addBands(gradmask);
  
        # Determine the 8 directions
        directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1);
        directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2));
        directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3));
        directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4));
        # The next 4 are the not() of the previous 4
        directions = directions.addBands(directions.select(0).Not().multiply(5));
        directions = directions.addBands(directions.select(1).Not().multiply(6));
        directions = directions.addBands(directions.select(2).Not().multiply(7));
        directions = directions.addBands(directions.select(3).Not().multiply(8));
  
        # Mask all values that are not 1-8
        directions = directions.updateMask(gradmask);
  
        # "collapse" the stack into a singe band image (due to masking, each pixel has just one value (1-8) in it's directional band, and is otherwise masked)
        directions = directions.reduce(ee.Reducer.sum());  
  
        sample_stats = sample_var.divide(sample_mean.multiply(sample_mean));
  
        #Calculate localNoiseVariance
        sigmaV = sample_stats.toArray().arraySort().arraySlice(0,0,5).arrayReduce(ee.Reducer.mean(), [0]);
  
        # Set up the 7*7 kernels for directional statistics
        rect_weights = ee.List.repeat(ee.List.repeat(0,7),3).cat(ee.List.repeat(ee.List.repeat(1,7),4));
  
        diag_weights = ee.List([[1,0,0,0,0,0,0], [1,1,0,0,0,0,0], [1,1,1,0,0,0,0], [1,1,1,1,0,0,0], [1,1,1,1,1,0,0], [1,1,1,1,1,1,0], [1,1,1,1,1,1,1]]);
  
        rect_kernel = ee.Kernel.fixed(7,7, rect_weights, 3, 3, False);
        diag_kernel = ee.Kernel.fixed(7,7, diag_weights, 3, 3, False);
  
        # Create stacks for mean and variance using the original kernels. Mask with relevant direction.
        dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1));
        dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1));
  
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)));
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)));
  
        # and add the bands for rotated kernels
        for i in range(1, 4):
            dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
            dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
            dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))
            dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))

  
        # "collapse" the stack into a single band image (due to masking, each pixel has just one value in it's directional band, and is otherwise masked)
        dir_mean = dir_mean.reduce(ee.Reducer.sum());
        dir_var = dir_var.reduce(ee.Reducer.sum());
  
        # A finally generate the filtered value
        varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))
  
        b = varX.divide(dir_var)
        result = dir_mean.add(b.multiply(img.subtract(dir_mean)))
  
        return result.arrayProject([0]).arrayFlatten([['sum']]).float()
    
    result = ee.ImageCollection(bandNames.map(inner)).toBands().rename(bandNames).copyProperties(image)
    
    return image.addBands(result, None, True) 

def leesigma_wrap(image,KERNEL_SIZE=30):
     #parameters
    Tk = ee.Image.constant(7) #number of bright pixels in a 3x3 window
    sigma = 0.9
    enl = 4
    target_kernel = 3
    bandNames = image.bandNames().remove('angle')
    #compute the 98 percentile intensity 
    z98 = ee.Dictionary(image.select(bandNames).reduceRegion(
            reducer= ee.Reducer.percentile([98]),
            geometry= image.geometry(),
            scale=10,
            maxPixels=1e13
        )).toImage()
    #select the strong scatterers to retain
    brightPixel = image.select(bandNames).gte(z98)
    K = brightPixel.reduceNeighborhood(ee.Reducer.countDistinctNonNull()
        ,ee.Kernel.square(target_kernel/2)) 
    retainPixel = K.gte(Tk)


    #compute the a-priori mean within a 3x3 local window
    #original noise standard deviation since the data is 5 look
    eta = 1.0/math.sqrt(enl) 
    eta = ee.Image.constant(eta)
    #MMSE applied to estimate the apriori mean
    reducers = ee.Reducer.mean().combine( \
                  reducer2= ee.Reducer.variance(), \
                  sharedInputs= True
                  )
    stats = image.select(bandNames).reduceNeighborhood( \
                  reducer= reducers, \
                      kernel= ee.Kernel.square(target_kernel/2,'pixels'), \
                          optimization= 'window')
    meanBand = bandNames.map(lambda bandName: ee.String(bandName).cat('_mean'))
    varBand = bandNames.map(lambda bandName:  ee.String(bandName).cat('_variance'))

    z_bar = stats.select(meanBand)
    varz = stats.select(varBand)

    oneImg = ee.Image.constant(1)
    varx = (varz.subtract(z_bar.abs().pow(2).multiply(eta.pow(2)))).divide(oneImg.add(eta.pow(2)))
    b = varx.divide(varz)
    xTilde = oneImg.subtract(b).multiply(z_bar.abs()).add(b.multiply(image.select(bandNames)))

    #step 3: compute the sigma range
    #Lookup table (J.S.Lee et al 2009) for range and eta values for intensity (only 4 look is shown here)
    LUT = ee.Dictionary({0.5: ee.Dictionary({'I1': 0.694,'I2': 1.385,'eta': 0.1921}),
                             0.6: ee.Dictionary({'I1': 0.630,'I2': 1.495,'eta': 0.2348}),
                             0.7: ee.Dictionary({'I1': 0.560,'I2': 1.627,'eta': 0.2825}),
                             0.8: ee.Dictionary({'I1': 0.480,'I2': 1.804,'eta': 0.3354}),
                             0.9: ee.Dictionary({'I1': 0.378,'I2': 2.094,'eta': 0.3991}),
                             0.95: ee.Dictionary({'I1': 0.302,'I2': 2.360,'eta': 0.4391})});

    #extract data from lookup
    sigmaImage = ee.Dictionary(LUT.get(str(sigma))).toImage()
    I1 = sigmaImage.select('I1')
    I2 = sigmaImage.select('I2')
    #new speckle sigma
    nEta = sigmaImage.select('eta')
    #establish the sigma ranges
    I1 = I1.multiply(xTilde)
    I2 = I2.multiply(xTilde)

    #step 3: apply MMSE filter for pixels in the sigma range
    #MMSE estimator
    mask = image.select(bandNames).gte(I1).Or(image.select(bandNames).lte(I2))
    z = image.select(bandNames).updateMask(mask)

    stats = z.reduceNeighborhood( \
                  reducer= reducers, \
                      kernel= ee.Kernel.square(KERNEL_SIZE/2,'pixels'), \
                          optimization= 'window')

    z_bar = stats.select(meanBand)
    varz = stats.select(varBand)


    varx = (varz.subtract(z_bar.abs().pow(2).multiply(nEta.pow(2)))).divide(oneImg.add(nEta.pow(2)))
    b = varx.divide(varz)
    #if b is negative set it to zero
    new_b = b.where(b.lt(0), 0)
    xHat = oneImg.subtract(new_b).multiply(z_bar.abs()).add(new_b.multiply(z))

    #remove the applied masks and merge the retained pixels and the filtered pixels
    xHat = image.select(bandNames).updateMask(retainPixel).unmask(xHat)
    output = ee.Image(xHat).rename(bandNames)
    return image.addBands(output, None, True)

def leesigma(KERNEL_SIZE):
    '''
    KERNEL_SIZE : 越小计算时间越长
    '''
    return partial(leesigma_wrap, KERNEL_SIZE=KERNEL_SIZE)

In [5]:
# 预定义函数
def make_dir(path):
    isExists = os.path.exists(path)
    # 判断结果
    if not isExists:
        os.makedirs(path)
        print(path + ' 创建成功')
    return path

def cut_geometry(geometry,block_size:float=0.05):
  '''
  block_size 定义方块大小(度)

  '''
  # 计算边界
  bounds = ee.List(ee.List(geometry.bounds().coordinates()).get(0))

  # 计算geometry的宽度和高度
  width = ee.Number(ee.List(bounds.get(2)).get(0)).subtract(ee.Number(ee.List(bounds.get(0)).get(0)))
  height = ee.Number(ee.List(bounds.get(2)).get(1)).subtract(ee.Number(ee.List(bounds.get(0)).get(1)))

  # 计算行和列的数量
  num_rows = height.divide(block_size).ceil()
  num_cols = width.divide(block_size).ceil()

  # 定义一个函数，用于生成512x512的方块
  def create_blocks(row, col):
      x_min = ee.Number(ee.List(bounds.get(0)).get(0)).add(col.multiply(block_size))
      y_min = ee.Number(ee.List(bounds.get(0)).get(1)).add(row.multiply(block_size))
      x_max = x_min.add(block_size)
      y_max = y_min.add(block_size)
      return ee.Geometry.Rectangle([x_min, y_min, x_max, y_max])

  # 生成方块列表
  block_list = []
  for row in range(num_rows.getInfo()):
      for col in range(num_cols.getInfo()):
          block = create_blocks(ee.Number(row), ee.Number(col))
          block_list.append(block)
  return block_list

def clip_AOI(col, AOI):
    return col.clip(AOI)

def time_difference(col, middle_date):
    time_difference = middle_date.difference(
        ee.Date(col.get('system:time_start')), 'days').abs()
    return col.set({'time_difference': time_difference})

def rm_nodata(col, AOI):
    # 将遮罩外的元素置-99,默认的遮罩为Nodata区域，并统计Nodata的数量
    allNone_num = col.select('VV_sigma0').unmask(-99).eq(-99).reduceRegion(
        **{
            'geometry': AOI,
            'reducer': ee.Reducer.sum(),
            'scale': 10,
            'maxPixels': 1e12,
            'bestEffort': True,
        }).get('VV_sigma0')
    return col.set({'numNodata': allNone_num})

def load_image_collection(aoi, start_date, end_date, middle_date,
                         dem=None,model=None,buffer=0,Filter=None,FilterSize=30):
    '''
    s1数据加载
    '''
    s1_col = (ee.ImageCollection("COPERNICUS/S1_GRD")
              .filter(ee.Filter.eq('instrumentMode', 'IW'))
                .filterBounds(aoi)
                .filterDate(start_date, end_date))
    # 图像滤波，可选
    if Filter:
        print('Begin Filter ...')
        if Filter == 'leesigma':
            s1_col = s1_col.map(leesigma(FilterSize))
        elif Filter == 'RefinedLee':
            s1_col = s1_col.map(RefinedLee)
        elif Filter == 'gammamap':
            s1_col = s1_col.map(gammamap(FilterSize))
        elif Filter == 'boxcar':
            s1_col = s1_col.map(boxcar(FilterSize))
        else:
          print('Wrong Filter')
    else:
        print('Without Filter')
        
    # 地形矫正，可选
    if dem and model:
        print('Begin Slop Correction ...')
        s1_col = slope_correction(s1_col, dem, model, buffer=buffer)
    else:
        Rename = lambda image:image.rename(['VV_sigma0', 'VH_sigma0','incAngle'])
        s1_col = s1_col.map(Rename)
        print('Without Slop Correction')
        
    # 裁剪并计算空洞数量
    s1_col = s1_col.map(partial(clip_AOI, AOI=aoi))
    s1_col = s1_col.map(partial(rm_nodata, AOI=aoi))

    # 计算与目标日期的差距
    s1_col = s1_col.map(partial(time_difference, middle_date=middle_date))
    
    # 删除角度波段select
    if model:
      pass
    else:
      s1_col = s1_col.map(lambda img: img.select(['VV_sigma0', 'VH_sigma0']))
    # 分离升降轨
    s1_descending = s1_col.filter(ee.Filter.eq(
        'orbitProperties_pass', 'DESCENDING'))
    s1_ascending = s1_col.filter(ee.Filter.eq(
        'orbitProperties_pass', 'ASCENDING'))
    
    # 如果所有图像都包含空洞则返回一张合成的图像，还需要重投影，否则计算会出错
    if s1_descending.filter(ee.Filter.eq('numNodata', 0)).size().getInfo() == 0:
      s1_descending = s1_descending.median().reproject(s1_descending.first().projection().getInfo()['crs'], None, 10).clip(aoi)
    else:
      s1_descending = s1_descending.filter(ee.Filter.eq('numNodata', 0))

    if s1_ascending.filter(ee.Filter.eq('numNodata', 0)).size().getInfo() == 0:
      s1_ascending = s1_ascending.median().reproject(s1_ascending.first().projection().getInfo()['crs'], None, 10).clip(aoi)
    else:
      s1_ascending = s1_ascending.filter(ee.Filter.eq('numNodata', 0))

    return s1_ascending, s1_descending

def Geemap_export(filename,collection=False,image=False,region=None,scale=10):
    if collection:
        # 这里导出时候使用region设置AOI，否则可能因为坐标系问题(未确定)，出现黑边问题
        geemap.ee_export_image_collection(collection,
                        out_dir=os.path.dirname(out_dir),
                        format = "ZIPPED_GEO_TIFF",region=region,scale=scale)
        print('collection save right')
    elif image:     
        geemap.ee_export_image(image,
                    filename=filename,
                    scale=scale, region=region, file_per_band=False,timeout=1500)
        print('image save right')
    else:
        print('Erro:collection && image must have one False')

In [6]:
# Kmeans
def afn_Kmeans(inputImg,defaultStudyArea,numberOfUnsupervisedClusters=2,nativeScaleOfImage=10,numPixels=1000):
    '''
    inputImg：任何图像，所有波段将用于聚类。
    numberOfUnsupervisedClusters：要创建的聚类数的可调参数。
    defaultStudyArea：训练样本的默认区域。
    nativeScaleOfImage：图像的本地比例尺。
    '''
    # Make a new sample set on the inputImg. Here the sample set is randomly selected spatially.并训练
    training = inputImg.sample(region=defaultStudyArea, scale=nativeScaleOfImage, numPixels=numPixels)
    cluster = ee.Clusterer.wekaKMeans(numberOfUnsupervisedClusters).train(training)
    # Now apply that clusterer to the raw image that was also passed in.
    toexport = inputImg.cluster(cluster)
    # 选择分类结果，并进行重命名
    clusterUnsup = toexport.select(0).rename('unsupervisedClass')
    return clusterUnsup

def afn_Cobweb(inputImg,defaultStudyArea,cutoff=0.004,nativeScaleOfImage=10,numPixels=1000):
    # 这个算法效果一般
    training = inputImg.sample(region=defaultStudyArea, scale=nativeScaleOfImage, numPixels=numPixels)
    cluster = ee.Clusterer.wekaCobweb(cutoff=cutoff).train(training)
    toexport = inputImg.cluster(cluster)
    # 选择分类结果，并进行重命名
    clusterUnsup = toexport.select(0).rename('unsupervisedClass')
    return clusterUnsup

def afn_Xmeans(inputImg,defaultStudyArea,numberOfUnsupervisedClusters=2,nativeScaleOfImage=10, numPixels=1000):
    training = inputImg.sample(region=defaultStudyArea, scale=nativeScaleOfImage, numPixels=numPixels)
    cluster = ee.Clusterer.wekaXMeans(maxClusters=numberOfUnsupervisedClusters).train(training)
    toexport = inputImg.cluster(cluster)
    clusterUnsup = toexport.select(0).rename('unsupervisedClass')
    return clusterUnsup

def afn_LVQ(inputImg,defaultStudyArea,numberOfUnsupervisedClusters=2,nativeScaleOfImage=10, numPixels=1000):
    training = inputImg.sample(region=defaultStudyArea, scale=nativeScaleOfImage,numPixels=numPixels)
    cluster = ee.Clusterer.wekaLVQ(numClusters=numberOfUnsupervisedClusters).train(training)
    toexport = inputImg.cluster(cluster)
    clusterUnsup = toexport.select(0).rename('unsupervisedClass')
    return clusterUnsup

def afn_CascadeKMeans(inputImg,defaultStudyArea,numberOfUnsupervisedClusters=2,nativeScaleOfImage=10, numPixels=1000):
    training = inputImg.sample(region=defaultStudyArea, scale=nativeScaleOfImage, numPixels=numPixels)
    cluster = ee.Clusterer.wekaCascadeKMeans(maxClusters=numberOfUnsupervisedClusters).train(training)
    toexport = inputImg.cluster(cluster)
    clusterUnsup = toexport.select(0).rename('unsupervisedClass')
    return clusterUnsup

def afn_SNIC(imageOriginal, SuperPixelSize=10, Compactness=1, Connectivity=4, NeighborhoodSize=20, SeedShape='square'):
    '''
    下面是对 afn_SNIC 函数的参数进行解释：
    imageOriginal: 要进行超像素分割的原始图像。这是必需的参数。
    SuperPixelSize: default=5意思是超像素大小，即生成的超像素的平均大小。通常情况下，该值取决于你要处理的图像的大小和分辨率。如果原始图像比较大，那么超像素的大小可以设置得较大，例如100或200像素。如果原始图像比较小，那么超像素的大小应设置得比较小，例如10或20像素。
    Compactness: default=1该参数控制着每个超像素的形态。它的值通常在[1,20]范围内取值。Compactness值越小，超像素类别的形状越规则，而 Compactness值越大，超像素的形状越灵活。
    Connectivity: default=8连通性参数，描述每个像素与周围像素的关系，这个参数通常设为8。
    NeighborhoodSize：瓦片邻域大小（为了避免瓦片边界伪影）。默认为2 * size。定义像素颜色在计算距离时考虑的领域大小。值越大，超像素越平滑，边缘越模糊；值越小，超像素分割边缘更为锐利。
    SeedShape：种子形状，可以是“square”或“hex”。
    '''
    theSeeds = ee.Algorithms.Image.Segmentation.seedGrid(SuperPixelSize, SeedShape)
    
    snic2 = ee.Algorithms.Image.Segmentation.SNIC(image=imageOriginal, 
                            size=SuperPixelSize, 
                            compactness=Compactness, 
                            connectivity=Connectivity, 
                            neighborhoodSize=NeighborhoodSize, 
                            seeds=theSeeds)
    theStack = snic2.addBands(theSeeds)
    return theStack

def afn_normalize_by_maxes(img,scale=10):
    '''
    注意：未进行重投影，max_\min_是地理坐标
    注意：使用使用
    img：要归一化的图像。
    bandMaxes：波段最大值。
    '''
    max_ = img.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=img.geometry(),
    scale=scale,
    maxPixels=1e12,
    bestEffort=True
    ).toImage().select(img.bandNames())

    min_ = img.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=img.geometry(),
        scale=scale,
        maxPixels=1e12,
        bestEffort=True
      ).toImage().select(img.bandNames()) #.reproject(img.select(0).projection().getInfo()['crs'], None, 10)

    return img.subtract(min_).divide(max_.subtract(min_))

In [7]:
def Cluster_math(method:str,img,bands:list,index:str,visual:bool,save:bool,Map,region=None):
  '''
  method ('Kmean','SNIC','SNIC_Kmean','LVQ','Xmeans','Cobweb','CascadeKMeans')
  '''
  img = img.select(bands)
  img = afn_normalize_by_maxes(img) # 归一化

  if method == 'Kmean' :
    result = afn_Kmeans(img,region)
  elif method == 'Cobweb':
    result = afn_Cobweb(img,region)
  elif method == 'Xmeans': 
    result = afn_Xmeans(img,region)
  elif method == 'LVQ':
    result = afn_LVQ(img,region)
  elif method == 'CascadeKMeans':
    result = afn_CascadeKMeans(img,region)
  elif method == 'SNIC':
    result = afn_SNIC(img)
    result = result.select(result.bandNames().removeAll(['clusters','seeds']))
    result = result.reproject(result.select(0).projection().getInfo()['crs'], None, 10) 
  elif method == 'SNIC_Kmean':
    result = afn_SNIC(img)
    # 默认舍弃cluster和seed    
    result = result.select(result.bandNames().removeAll(['clusters','seeds']))
    result = result.reproject(result.select(0).projection().getInfo()['crs'], None, 10) 
    result0 = afn_Kmeans(result,region)          # 原始图像不参与
    result1 = afn_Kmeans(result.addBands(img),region)   # 原始图像参与    .unmask(10)

  if visual:
    Map.centerObject(img.geometry(), zoom=15)
    Map.addLayer(img.select(0), {'min':0,'max':1}, 'Origin')#
    if method in ['Kmean','Cobweb','Xmeans','LVQ','CascadeKMeans']:
      Map.addLayer(result.randomVisualizer(), {}, method)
    elif method == 'SNIC':
      Map.addLayer(result.randomVisualizer(), {}, method)
    elif method == 'SNIC_Kmean':
      Map.addLayer(result0.randomVisualizer(), {}, 'SNIC_Kmean_NoOrigin')
      Map.addLayer(result1.randomVisualizer(), {}, 'SNIC_Kmean_YesOrigin')
    else:
      print('Please check your method str')

  if save:
    if method == 'SNIC_Kmean':
      Geemap_export(filename=index+'NoOrigin'+method+'.tif',collection=False,image=result0,region=region,scale=10)
      Geemap_export(filename=index+'YesOrigin'+method+'.tif',collection=False,image=result1,region=region,scale=10)
    else:
      Geemap_export(filename=index+method+'.tif',collection=False,image=result,region=region,scale=10)
    pass
  if method == 'SNIC_Kmean':
    return Map,result0.addBands(result1)
  else:
    return Map,result

In [8]:
def get_histogram(Image,region=None):
  histogram0 = Image.reduceRegion(
    reducer=ee.Reducer.histogram(1000),
    geometry=region,
    scale=10,
    maxPixels=1e12,
    bestEffort=True )#自动平衡scale减少计算量
  histogram = histogram0.get(Image.bandNames().get(0))
  return histogram

def afn_otsu(histogram):
  # 各组频数
  counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
  # 各组的值
  means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
  # 组数
  size = means.length().get([0])
  # 总像元数量
  total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
  # 所有组的值之和
  sum_ = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
  # 整幅影像的均值
  mean_ = sum_.divide(total)
  # 与组数相同长度的索引
  indices = ee.List.sequence(1, size)

  def calc_bss(i,sum_,mean_):
    '''穷举法计算类内方差''' 
    aCounts = counts.slice(0, ee.Number(i)) 
    # 从i分割为两类A、B 计算A方差
    aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0]) 
    aMeans = means.slice(0, ee.Number(i)) 
    # 类别A均值
    aMean = aMeans.multiply(aCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(aCount) 
    bCount = total.subtract(aCount) 
    # 类别B均值
    bMean = sum_.subtract(aCount.multiply(aMean)).divide(bCount)
    # 类间方差
    return aCount.multiply(aMean.subtract(mean_).pow(2)).add(bCount.multiply(bMean.subtract(mean_).pow(2)))

  # 计算类内方差
  bss = indices.map(partial(calc_bss,sum_=sum_,mean_=mean_))
  # 排序选出最适阈值
  return means.sort(bss).get([-1])

from skimage.filters import threshold_minimum
import numpy as np 

def afn_histPeak(img,region=None):
  '''
  采用skimage计算双峰
  参考论文：An analysis of histogram-based thresholding algorithms
  The analysis of cell images
  '''
  img = afn_normalize_by_maxes(img)
  img_numpy = geemap.ee_to_numpy(img,region=region)  #region必须是矩形
  threshold = threshold_minimum(img_numpy)
  return threshold

def Bandmath(method:str,img,index:str,visual:bool,save:bool,Map,region=None):
  '''
  method = ['otsu','threshould','histogram']
  img: 仅单波段图像
  '''
  assert len(img.bandNames().getInfo()) == 1 , '图像波段数必须唯一'
  img = afn_normalize_by_maxes(img) # 归一化

  if method == 'otsu':
    histogram = get_histogram(img,region=region)
    Threshould_value = afn_otsu(histogram)
    result = img.select(0).gt(Threshould_value)  #
    print('Threshould value is {}'.format(Threshould_value.getInfo()))  

  elif method == 'histPeak':
    Threshould_value = afn_histPeak(img,region=region)
    result = img.gt(Threshould_value)
    print('Threshould value is {}'.format(Threshould_value))

  if visual:
    Map.centerObject(img.geometry(), zoom=15)
    Map.addLayer(img, {'min':0,'max':1}, 'Origin')#
    if method in ['otsu','histPeak']:
      Map.addLayer(result.randomVisualizer(), {}, method)
    else:
      print('Wrong visual! Please check your method str')

  if save:
    if method in ['otsu','histPeak']:
      Geemap_export(filename=index+method+'.tif',collection=False,image=result,region=region,scale=10)
    else:
      print('Wrong save! Please check your method str')

  return result

In [9]:
style = {'description_width': 'initial'}

# 指定第一个shp，感觉仅为了提供省份的名称
Start_Date = widgets.Text(
    description='Start_Date:', value='2019-07-01', width=200, style=style
)
# 指定第二个shp，用于计算方位
End_Date = widgets.Text(
    description='End_Date:', value='2019-08-30', width=200, style=style
)

Terrain = widgets.Dropdown(
    description='Terrain correction:',
    options=[
        'volume',
        'surface',
        None,
    ],
    value=None,
    style=style,
)

Filter = widgets.Dropdown(
    description='SAR Filter:',
    options=[
        'RefinedLee',
        'leesigma',
        'gammamap',
        'boxcar'
    ],
    value='RefinedLee',
    style=style,
)

Method = widgets.Dropdown(
    description='Method:',
    options=[
        'otsu',
        'histPeak',
        'Kmean',
        'SNIC_Kmean',
        'LVQ',
        'Xmeans',
        'CascadeKMeans'

    ],
    value='SNIC_Kmean',
    style=style,
)

# 展示button
widgets.HBox([Start_Date, End_Date, Terrain, Filter,Method])

In [10]:
Number = widgets.IntSlider(
    min=0, max=961, value=0, description='Select the GL Index', width=400, style=style)
# 下载chart
download_widget = widgets.Checkbox(
    value=False, description='Download classified image', style=style
)
widgets.HBox([Number,download_widget])

In [11]:
submit = widgets.Button(
    description='Submit', button_style='primary', tooltip='Click me', style=style
)
# widgets.HBox([submit])

In [12]:
Map = geemap.Map(basemap='HYBRID') # 
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [13]:
# Click event handler
def submit_clicked(b):
  # 选择一个时间点的影像，注意SAR和光学影像的时间点已经不一样了
  # 需要放宽一些时间限制，否则可能会因为天气影响、卫星维护等问题出现数据缺失
  START_DATE  = ee.Date(Start_Date.value)
  END_DATE   = ee.Date(End_Date.value)
  TIME_LEN   = END_DATE.difference(START_DATE, 'days').abs()
  MIDDLE_DATE  = START_DATE.advance(TIME_LEN.divide(ee.Number(2)).int(),'days')
  dem = ee.Image('USGS/SRTMGL1_003') # paths to dem 
  # models = ['volume', 'surface', None]     # 地形矫正模型

  # 选择一个点做实验
  i=Number.value #394(面积大) #53 #11(影像都用nodata) #149(坐标系问题)

  AOI_point = ee.Feature.geometry(Glacial_lake_2015C_CentriodList.get(i))
  AOI_area = ee.Feature.geometry(Glacial_lake_2015A_GeoList.get(i))
  AOI = ee.Feature.geometry(Glacial_lake_2015R_RectangleList.get(i))

  # 计算面积
  AOI_area_area = AOI_area.area().divide(ee.Number(1000*1000)).getInfo()
  # 缩小
  if AOI_area_area < 1: 
    AOI_area_buffer = AOI_area.buffer(distance=AOI_area_area*-300)
  else:
    AOI_area_buffer = AOI_area.buffer(distance=-400)
    
  # 扩大包络矩形AOI,保证背景像素占比最大
  if AOI_area_area < 1: 
    AOI_buffer = AOI.buffer(distance=300)
  else:
    AOI_buffer = AOI.buffer(distance=400)

  # 在这个点采用buffer裁剪文件，展示SAR，去除了包含空值的影像
  s1_ascending,s1_descending = load_image_collection(AOI_buffer,START_DATE,END_DATE,MIDDLE_DATE,
                            dem=dem,model=Terrain.value,Filter=Filter.value,FilterSize=30)

  # 全包含空值条件下，将图像集合合并为一张图像进行处理
  if type(s1_ascending) == ee.imagecollection.ImageCollection:
    s1_single_a = s1_ascending.filter(ee.Filter.eq('time_difference', 
            s1_ascending.aggregate_min('time_difference'))).first().set({'synthesis': 0})
  else:
    s1_single_a = s1_ascending.set({'synthesis': 1})
  if type(s1_descending) == ee.imagecollection.ImageCollection:
    s1_single_d = s1_descending.filter(ee.Filter.eq('time_difference', 
            s1_descending.aggregate_min('time_difference'))).first().set({'synthesis': 0})
  else:
    s1_single_d = s1_descending.set({'synthesis': 1})

  # 加入条件空值，保证在空值情况下图像能够正常相加
  condition = s1_single_d.mask().clip(AOI_buffer)
  s1_unit_mean = s1_single_a.where(condition, s1_single_a.add(s1_single_d).divide(2)) #转为均值
  s1_unit_max = s1_single_a.where(condition, s1_single_a.max(s1_single_d))
  # s1_unit_add = s1_single_a.where(condition, s1_single_a.add(s1_single_d))

  s1_unit_all = s1_unit_mean.addBands(s1_unit_max)
  if Method.value in ['Kmean','SNIC_Kmean','LVQ','Xmeans','CascadeKMeans']:
    result= Cluster_math(method='SNIC_Kmean',img=s1_unit_all,bands=[0,1],index='',visual=True,save=False,region=AOI_buffer,Map=Map)  # 如果迭代多个影像可以采用调整index
  elif Method.value in ['otsu','histPeak']:
    result = Bandmath(method='histPeak',img=s1_unit_all.select(0),index='',visual=True,save=False,region=AOI,Map=Map)

  if download_widget.value == True:
    pass

submit.on_click(submit_clicked)

In [14]:
full_widget = widgets.VBox(
    [
        widgets.HBox([Start_Date, End_Date, Terrain, Filter]),
        widgets.HBox([Method,Number,download_widget]),
        submit,
    ]
)

full_widget

Begin Filter ...
Without Slop Correction
Begin Filter ...
Without Slop Correction
Begin Filter ...
Without Slop Correction
Begin Filter ...
Without Slop Correction
